# Feature embedding

In [4]:
import numpy as np

band_powers = np.load('/home/vincent/AAA_projects/MVCS/Neuroscience/Analysis/Spectral Analysis/BandPowers_x.npy', allow_pickle=True).item()

In [6]:
import os
os.environ['LD_LIBRARY_PATH'] = '/home/vincent/anaconda3/lib/:' + os.environ.get('LD_LIBRARY_PATH', '')


In [10]:
import numpy as np
import torch

# Load the dictionary from the numpy file
band_powers = np.load('/home/vincent/AAA_projects/MVCS/Neuroscience/Analysis/Spectral Analysis/BandPowers_x.npy', allow_pickle=True).item()

# Convert each channel's dictionary of band powers into a single tensor
band_power_tensors = []
for channel, bands in band_powers.items():
    channel_data = [torch.tensor(bands[band_name], dtype=torch.float32) for band_name in bands]
    channel_tensor = torch.stack(channel_data).flatten()  # Flattening all band powers for the channel into one tensor
    band_power_tensors.append(channel_tensor)

band_power_matrix = torch.stack(band_power_tensors).transpose(0, 1)  # shape: [timepoints, flattened channel data]

# Define the device (either 'cpu' or 'cuda')
device = 'cuda'  # or 'cpu' if you want to use the CPU

# Move the tensor to the specified device
band_power_matrix = band_power_matrix.to(device)

# Save the processed tensor to a file
save_path = '/home/vincent/AAA_projects/MVCS/Neuroscience/Models/Feature Embedding/band_power_matrix.pth'
torch.save(band_power_matrix, save_path)


In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

# Define the device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load band_power_matrix
path_to_data = '/home/vincent/AAA_projects/MVCS/Neuroscience/Models/Feature Embedding/band_power_matrix.pth'
band_power_matrix = torch.load(path_to_data)
band_power_matrix = band_power_matrix.to(device)  # Move the data to the same device as the model

class EmbeddingNet(nn.Module):
    def __init__(self, input_size):
        super(EmbeddingNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x

model = EmbeddingNet(band_power_matrix.shape[1]).to(device)

if torch.cuda.device_count() > 1:
    print("Using", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)

# Set model to evaluation mode if not training
model.eval()

# Create a dataset from the tensor
dataset = TensorDataset(band_power_matrix)

# Decide on a batch size
batch_size = 256

# Create a data loader
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Loop over mini-batches
total_batches = len(dataloader)
for i, batch in enumerate(dataloader):
    inputs = batch[0].to(device)
    embedded_band_power_batch = model(inputs)
    print(f"Processed batch {i+1}/{total_batches}")

print(model)
# Save the model
model_save_path = "/home/vincent/AAA_projects/MVCS/Neuroscience/Models/bandpowers_embedding_net.pth"
torch.save(model.state_dict(), model_save_path)


Using 2 GPUs!
Processed batch 1/82574
Processed batch 2/82574
Processed batch 3/82574
Processed batch 4/82574
Processed batch 5/82574
Processed batch 6/82574
Processed batch 7/82574
Processed batch 8/82574
Processed batch 9/82574
Processed batch 10/82574
Processed batch 11/82574
Processed batch 12/82574
Processed batch 13/82574
Processed batch 14/82574
Processed batch 15/82574
Processed batch 16/82574
Processed batch 17/82574
Processed batch 18/82574
Processed batch 19/82574
Processed batch 20/82574
Processed batch 21/82574
Processed batch 22/82574
Processed batch 23/82574
Processed batch 24/82574
Processed batch 25/82574
Processed batch 26/82574
Processed batch 27/82574
Processed batch 28/82574
Processed batch 29/82574
Processed batch 30/82574
Processed batch 31/82574
Processed batch 32/82574
Processed batch 33/82574
Processed batch 34/82574
Processed batch 35/82574
Processed batch 36/82574
Processed batch 37/82574
Processed batch 38/82574
Processed batch 39/82574
Processed batch 40/8

In [ ]:
embedded_band_power = model(band_power_matrix)
